In [41]:
# !pip install catboost==0.15.2
# !pip install tqdm
# !pip install xlrd
import numpy as np
import pandas as pd
import catboost as cbt
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
import gc
import math
import time
from tqdm import tqdm
import datetime
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [42]:
train = pd.read_csv('/home/aistudio/data/data11870/first_round_training_data.csv')
test = pd.read_csv('/home/aistudio/data/data11870/first_round_testing_data.csv')
submit = pd.read_csv('/home/aistudio/data/data11870/submit_example.csv')
data = train.append(test).reset_index(drop=True)
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
data['label'] = data['Quality_label'].map(dit)

In [43]:
fea_list = [i for i in test.columns if i not in ['Group']] 
cat_list = []
lbl = LabelEncoder()
for i in tqdm(fea_list):
    if train['{}'.format(i)].nunique() < 6000:
        cat_list.append('{}'.format(i))

(12000, 23)

In [1]:
feature_name = [i for i in data.columns if i not in ['Attribute1','Attribute10', 'Attribute2', 'Attribute3', 'Attribute4', 'Attribute5',
 'Attribute6', 'Attribute7', 'Attribute8', 'Attribute9', 'Quality_label','Group','label']]
tr_index = ~data['label'].isnull()
X_train = data[tr_index][feature_name].reset_index(drop=True)
y = data[tr_index]['label'].reset_index(drop=True).astype(int)
X_test = data[~tr_index][feature_name].reset_index(drop=True)

print(X_train.shape,X_test.shape)
oof = np.zeros((X_train.shape[0],4))
prediction = np.zeros((X_test.shape[0],4))
seeds = [19970412, 2019 * 2 + 1024, 4096, 2048, 1024]
num_model_seed = 2
for model_seed in range(num_model_seed):
    print(model_seed + 1)
    oof_cat = np.zeros((X_train.shape[0],4))
    prediction_cat=np.zeros((X_test.shape[0],4))
    skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
        print(index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        gc.collect()
        cbt_model = cbt.CatBoostClassifier(iterations=100000,learning_rate=0.01,verbose=300,max_depth=7,#eval_metric='Accuracy',
                                        early_stopping_rounds=1000,task_type='GPU',cat_features=cat_list,
                                        loss_function='MultiClass')
        cbt_model.fit(train_x, train_y ,eval_set=(test_x,test_y))
        oof_cat[test_index] += cbt_model.predict_proba(test_x)
        prediction_cat += cbt_model.predict_proba(X_test)/5
        gc.collect()
    oof += oof_cat / num_model_seed
    prediction += prediction_cat / num_model_seed
    print('logloss',log_loss(pd.get_dummies(y).values, oof_cat))
    print('ac',accuracy_score(y, np.argmax(oof_cat,axis=1)))
print('logloss',log_loss(pd.get_dummies(y).values, oof))
print('ac',accuracy_score(y, np.argmax(oof,axis=1)))

In [47]:
sub = test[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]
for i, f in enumerate(prob_cols):
    sub[f] = prediction[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()
sub.to_csv("submission.csv",index=False)